# setup

In [1]:
import torch
import torch.nn as nn
import os

In [2]:
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

# from configs.baseline import LoadDataConfig
from configs.fake import LoadDataConfig
from configs.moe import MoE_cnn_args
from data.load_data import LoadData
from models.moe import ResnetMoE
from utils import train, eval, plot_log, export

# init

In [3]:
model_label = 'moe'

In [4]:
loader_config = LoadDataConfig()
moe_config = MoE_cnn_args()

In [5]:
dataloader = LoadData(**loader_config.__dict__)
# model = ResnetMoE(**moe_config.__dict__)
model = torch.load('output/pretrained_moe.pt')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EPOCHS = 5 + 1

--> WARNING: USING RANDOM FAKE DATA!!


# train

In [6]:
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

In [7]:
log = []
for epoch in range(5, EPOCHS):
    train_dl, val_dl = dataloader.get_train_dataloader(), dataloader.get_val_dataloader()

    train_log = train(model, train_dl, optimizer, criterion, device)
    val_log = eval(model, val_dl, criterion, device)
    plot_log(train_log, val_log, epoch = epoch)
    export(model, model_label, epoch)

100%|██████████| 1/1 [00:00<00:00, 30.57it/s]


exporting partial model at epoch 5


In [8]:
torch.save(model, 'output/{}.pt'.format(model_label))